# exp032_trial
tfidf_nameのngram変更

In [23]:
MODE = 'local_train'
#MODE = 'kaggle_inference'

In [24]:
exp_name = 'exp032'
memo = '1st改善'

In [25]:
import os
import sys
import gc

if MODE == 'local_train':
    sys.path.append('/home/kaggler/.local/lib/python3.8/site-packages')
    from dotenv import load_dotenv
    load_dotenv
    sys.path.append(os.getenv('UTILS_PATH'))
    import line_notify
    import slack_notify
    
if MODE == "kaggle_inference":
    from cuml import ForestInference
    import treelite
    
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.style.use("ggplot")
import seaborn as sns
from sklearn.metrics.pairwise import haversine_distances
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb
import itertools
from scipy.spatial.distance import canberra
from sklearn.neighbors import KNeighborsRegressor
import functools
import multiprocessing
import Levenshtein
import difflib
import pickle
from tqdm import tqdm
%load_ext Cython

from transformers import DistilBertModel, DistilBertTokenizer
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
import torch

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [26]:
# directry_setting
if MODE == 'local_train':
    INPUT_DIR = os.getenv('INPUT_DIR')
    OUTPUT_DIR = os.getenv('OUTPUT_DIR')
    MODEL_DIR = os.getenv('OUTPUT_DIR')
    BERT_MODEL = "distilbert-base-multilingual-cased"
    #os.makedirs(OUTPUT_DIR + exp_name, exist_ok=True)

elif MODE == 'kaggle_inference':
    INPUT_DIR = '/kaggle/input/foursquare-location-matching/'
    OUTPUT_DIR = './'
    MODEL_DIR = f'../input/fs{exp_name}/'
    BERT_MODEL = "../input/distilbertbaseuncased"

In [27]:
# CONFIG
SEED = 42
N_NEIGHBORS = 10
N_SPLITS = 5
PROB_TH = 0.5
MAX_LEN = 32
BS = 512
NW = 2
SVD_N_COMP = 50

In [28]:
def preprocess(df):
    columns = ['id', 'name', 'address', 'city', 'state',
        'zip', 'country', 'url', 'phone', 'categories']
    for c in columns:
        if c != "id":
            df[c] = df[c].astype(str).str.lower()

    df[["latitude", "longitude"]] = np.deg2rad(df[["latitude", "longitude"]])
    
    return df

In [29]:
from cuml.feature_extraction.text import TfidfVectorizer as TfidfVectorizer_gpu
import cudf, cuml, cupy
from cuml.neighbors import NearestNeighbors as NearestNeighbors_gpu

In [30]:
def extract_candidate(df):
    dfs = []
    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df.reset_index(drop=True)
        
        knn = KNeighborsRegressor(n_neighbors=min(len(country_df), N_NEIGHBORS), 
                                    metric='haversine', n_jobs=-1)
        knn.fit(country_df[['latitude','longitude']], country_df.index)
        dists, nears = knn.kneighbors(country_df[['latitude','longitude']], return_distance=True)
        
        k = min(len(country_df), N_NEIGHBORS)
        country_df['match_id'] = country_df['id'].values[nears[:, :k]].tolist()
        country_df['d_near'] = dists[:, :k].tolist()
        country_df = country_df.explode(['match_id','d_near'])
        country_df = country_df.loc[country_df['id'] != country_df['match_id']].copy()
        dfs.append(country_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [31]:
def extract_candidate_tfidf_name(df):
    dfs = []
    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df.reset_index(drop=True)
        
        model = TfidfVectorizer_gpu(stop_words='english', binary=True)
        text_embeddings = model.fit_transform(cudf.from_pandas(country_df["name"]))

        model = NearestNeighbors_gpu(n_neighbors=min(len(country_df), N_NEIGHBORS), algorithm="brute")
        model.fit(text_embeddings)
        dists, nears = model.kneighbors(text_embeddings)
        
        k = min(len(country_df), N_NEIGHBORS)
        country_df['match_id'] = country_df['id'].values[nears[:, :k].get()].tolist()
        country_df['d_near'] = dists[:, :k].tolist()
        country_df = country_df.explode(['match_id','d_near'])
        country_df = country_df.loc[country_df['id'] != country_df['match_id']].copy()
        dfs.append(country_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [32]:
def extract_candidate_tfidf_name2(df):
    dfs = []
    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df.reset_index(drop=True)
        
        model = TfidfVectorizer(stop_words='english')
        text_embeddings = model.fit_transform(country_df["name"].tolist())

        model = NearestNeighbors_gpu(n_neighbors=min(len(country_df), N_NEIGHBORS), algorithm="brute")
        model.fit(text_embeddings)
        dists, nears = model.kneighbors(text_embeddings)
        
        k = min(len(country_df), N_NEIGHBORS)
        country_df['match_id'] = country_df['id'].values[nears[:, :k]].tolist()
        country_df['d_near'] = dists[:, :k].tolist()
        country_df = country_df.explode(['match_id','d_near'])
        country_df = country_df.loc[country_df['id'] != country_df['match_id']].copy()
        dfs.append(country_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [33]:
def extract_candidate_tfidf_name3(df):
    dfs = []
    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df.reset_index(drop=True)
        
        model = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
        text_embeddings = model.fit_transform(country_df["name"].tolist())

        model = NearestNeighbors_gpu(n_neighbors=min(len(country_df), N_NEIGHBORS), algorithm="brute")
        model.fit(text_embeddings)
        dists, nears = model.kneighbors(text_embeddings)
        
        k = min(len(country_df), N_NEIGHBORS)
        country_df['match_id'] = country_df['id'].values[nears[:, :k]].tolist()
        country_df['d_near'] = dists[:, :k].tolist()
        country_df = country_df.explode(['match_id','d_near'])
        country_df = country_df.loc[country_df['id'] != country_df['match_id']].copy()
        dfs.append(country_df)
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [34]:
def add_orgin_data(df, org_df):
    df = df.merge(org_df.add_prefix('match_'), on='match_id')
    df = df.reset_index(drop=True)
    return df

In [35]:
# https://www.kaggle.com/code/columbia2131/foursquare-iou-metrics
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame, org_data):
    scores = []
    id2poi = get_id2poi(org_data)
    poi2ids = get_poi2ids(org_data)
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def calc_max_score(tr_data, org_data):
    train_candidate = pd.DataFrame()
    train_candidate['id'] = org_data['id'].unique()
    train_candidate['matches'] = org_data['id'].unique()
    idx = tr_data['point_of_interest']==tr_data['match_point_of_interest']
    train_match = tr_data.loc[idx].groupby('id')['match_id'].apply(list).map(" ".join).reset_index()
    train_match.columns = ['id','candidates']
    train_candidate = train_candidate.merge(train_match, on = 'id', how = 'left')
    idx = ~train_candidate['candidates'].isna()
    train_candidate.loc[idx, "matches"] += " " + train_candidate.loc[idx, "candidates"]
    score = get_score(train_candidate, org_data)
    print('1st_stage_max_score : ' + '{:.5f}'.format(score))
    return score

In [36]:
train_origin = pd.read_csv(INPUT_DIR + "train.csv")
train_origin = preprocess(train_origin)

# trainデータの分割
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(train_origin, train_origin['point_of_interest'], train_origin['point_of_interest'])):
    train_origin.loc[val_idx, "set"] = i

In [37]:
conditions = []
lens = []
scores = []

In [39]:
condition = "元"
N_NEIGHBORS = 10
dist_df = extract_candidate(train_origin[train_origin["set"]==0])
namesim_df = extract_candidate_tfidf_name(train_origin[train_origin["set"]==0])
train = pd.concat([dist_df, namesim_df])
train = train.drop_duplicates(subset=["id", "match_id"])
train = add_orgin_data(train, train_origin)
score = calc_max_score(train, train_origin)
conditions.append(condition)
lens.append(len(train))
scores.append(score)
print(condition, len(train), score)

100%|██████████| 210/210 [00:44<00:00,  4.73it/s]


1st_stage_max_score : 0.79885
元 10010699 0.7988484468223355


In [40]:
condition = "gpu使わない"
N_NEIGHBORS = 10
dist_df = extract_candidate(train_origin[train_origin["set"]==0])
namesim_df = extract_candidate_tfidf_name2(train_origin[train_origin["set"]==0])
train = pd.concat([dist_df, namesim_df])
train = train.drop_duplicates(subset=["id", "match_id"])
train = add_orgin_data(train, train_origin)
score = calc_max_score(train, train_origin)
conditions.append(condition)
lens.append(len(train))
scores.append(score)
print(condition, len(train), score)

100%|██████████| 210/210 [00:26<00:00,  7.96it/s]


1st_stage_max_score : 0.79885
gpu使わない 10010525 0.7988471385620681


In [41]:
condition = "gpu使わない ngram=(1,2)"
N_NEIGHBORS = 10
dist_df = extract_candidate(train_origin[train_origin["set"]==0])
namesim_df = extract_candidate_tfidf_name3(train_origin[train_origin["set"]==0])
train = pd.concat([dist_df, namesim_df])
train = train.drop_duplicates(subset=["id", "match_id"])
train = add_orgin_data(train, train_origin)
score = calc_max_score(train, train_origin)
conditions.append(condition)
lens.append(len(train))
scores.append(score)
print(condition, len(train), score)

100%|██████████| 210/210 [00:29<00:00,  7.12it/s]


1st_stage_max_score : 0.79451
gpu使わない ngram=(1,2) 10074033 0.7945136874622389
